# V3 Auto Finance Table

In [35]:
# Import Libraries
########################################
#         Data Import from Sheets      #
########################################
import gspread
import pandas as pd
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.message import EmailMessage
import os
import mimetypes
from pathlib import Path
import json
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
# Potential Future Use
# email_username = os.getenv("EMAIL_USERNAME")
# email_password = os.getenv("EMAIL_PASSWORD")


In [37]:
# Credentials for Importing Google Sheets Data
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv('TRY'), scope)
json_keyfile_dict = json.loads(os.getenv("JSON_KEYFILE_DICT"))
creds = ServiceAccountCredentials.from_json_keyfile_dict(json_keyfile_dict, scope)
client = gspread.authorize(creds)
spreadsheet_key = os.getenv("SPREADSHEET_KEY2")
spreadsheet = client.open_by_key(spreadsheet_key)

In [38]:
# Import Expenses Data
sheet_title = 'Transactions'  
sheet = spreadsheet.worksheet(sheet_title)
expenses_data = sheet.get_all_records()
expenses_data = pd.DataFrame(expenses_data)

# Clean Time data
expenses_data["Date"] = expenses_data["Date"].astype(str).str.strip()

expenses_data["Date"] = pd.to_datetime(
    expenses_data["Date"],
    format="mixed",            
    utc=True,                  
    dayfirst=False          
)

expenses_data["Date"] = expenses_data["Date"].dt.tz_convert(None)

# Remove Money Hold if applicable
expenses_data = expenses_data[expenses_data['Category'] != "Hold"]

In [39]:
# Import Income Data
sheet_title = 'Income'  
sheet = spreadsheet.worksheet(sheet_title)
income_data = sheet.get_all_records()
income_data = pd.DataFrame(income_data)

# Clean Time data
income_data["Date"] = income_data["Date"].astype(str).str.strip()

income_data["Date"] = pd.to_datetime(
    income_data["Date"],
    format="mixed",            
    utc=True,                  
    dayfirst=False          
)

income_data["Date"] = income_data["Date"].dt.tz_convert(None)

In [40]:
import pandas as pd

# -- 1) Prepare expenses DataFrame with and without outliers --

# Working copy
df = expenses_data.copy().rename(columns={'Amount': 'expenses'})
df['Year-Month'] = df['Date'].dt.to_period('M')

monthly_expenses_all = (
    df
    .groupby('Year-Month')['expenses']
    .sum().round(2)
    .reset_index()
)

df_no = expenses_data[expenses_data['Outlier'] != 1].copy().rename(columns={'Amount': 'expenses_no_outlier'})
df_no['Year-Month'] = df_no['Date'].dt.to_period('M')

monthly_expenses_no_outlier = (
    df_no
    .groupby('Year-Month')['expenses_no_outlier']
    .sum().round(2)
    .reset_index()
)

# -- 2) Prepare income DataFrame and total income --

df_inc = income_data.copy().rename(columns={'Amount': 'income'})
df_inc['Year-Month'] = df_inc['Date'].dt.to_period('M')

monthly_income_all = (
    df_inc
    .groupby('Year-Month')['income']
    .sum().round(2)
    .reset_index()
)

# -- 3) Pivot expenses by category --

expense_categories = [
    "Groceries",
    "Restaurant/Fast Food",
    "Gas",
    "House Supplies",
    "Charity",
    "Miscellaneous/Gifts",
    "Leisure",
    "Clothing",
    "Mortgage/Rent",
    "Health",
    "Utilities Energy",
    "Utilities Gas",
    "Utilities Water",
    "Utilities Internet",
    "Utilities Phone",
    "Car Maintenance",
    "Car Insurance",
    "Car Payments",
    "Flights",
    "Vacation",
    "Chat GPT",
    "Gym",
    "Taxi",
    "Legal",
    "Taxes",
    "Loan Payment",
    "Education",
    "Loss",
    "Wedding"
]

df_cat = df[df['Category'].isin(expense_categories)]
monthly_expenses_by_cat = (
    df_cat
    .groupby(['Year-Month', 'Category'])['expenses']
    .sum().round(2)
    .unstack(fill_value=0)
    .reset_index()
)

# -- 4) Pivot income by category --

# You can list specific income categories here, or just pivot all that exist:
income_categories = df_inc['Category'].unique().tolist()

monthly_income_by_cat = (
    df_inc
    .groupby(['Year-Month', 'Category'])['income']
    .sum().round(2)
    .unstack(fill_value=0)
    .reset_index()
)

# -- 5) Merge everything together --

table = (
    monthly_expenses_no_outlier
    .merge(monthly_expenses_all,    on='Year-Month')
    .merge(monthly_income_all,      on='Year-Month', how='left')
    .merge(monthly_expenses_by_cat, on='Year-Month', how='left')
    .merge(monthly_income_by_cat,   on='Year-Month', how='left')
    .fillna(0)
)

# -- 6) Compute plus_minus and running net_worth --

table['plus_minus'] = (table['income'] - table['expenses']).round(2)
table['net_worth']  = table['plus_minus'].cumsum().round(2)

# -- 7) Append an “AVERAGE” row (including all category columns) --

# Gather all category columns
all_cat_cols = expense_categories + income_categories

# Compute means for each
means = {col: table[col].mean().round(2) for col in ['expenses_no_outlier','expenses','income','plus_minus'] + all_cat_cols}

avg_row = {'Year-Month': 'AVERAGE', **means, 'net_worth': pd.NA}
table = pd.concat([table, pd.DataFrame([avg_row])], ignore_index=True)

table['expenses_outlier'] = table['expenses'] - table['expenses_no_outlier']

# -- 8) Reorder columns as desired --

cols_order = (
    ['Year-Month','expenses_no_outlier','expenses','income','plus_minus','net_worth', 'expenses_outlier', 'Year-Month']
    + expense_categories
    + ['Year-Month'] 
    + income_categories
)
table = table[cols_order]


C:\Users\lohan\AppData\Local\Temp\ipykernel_24664\3697861431.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  table = pd.concat([table, pd.DataFrame([avg_row])], ignore_index=True)


In [41]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


# 2. Convert to HTML with inline CSS for gridlines
html_table = table.to_html(index=False, border=0)
styled_html = f"""
<html>
  <head>
    <style>
      table, th, td {{
        border: 1px solid black;
        border-collapse: collapse;
        padding: 4px;
      }}
      th {{ background-color: #f2f2f2; }}
    </style>
  </head>
  <body>
    <p>Hi there,</p>
    <p>See the table below:</p>
    {html_table}
    <p>Best,<br>Your Python Script</p>
  </body>
</html>
"""

# 3. Build the email message
msg = MIMEMultipart('alternative')
msg['Subject'] = "Monthly Expenses Overall"
msg['From']    = "lorossiprojects@gmail.com"
msg['To']      = "lohan.rossi@hotmail.com"

# Attach the HTML part
msg.attach(MIMEText(styled_html, 'html'))

# 4. Send via SMTP (example: Gmail SMTP server)
smtp_server = "smtp.gmail.com"
smtp_port   = 587
smtp_user   = "lorossiprojects@gmail.com"
smtp_pass   = "esso ebao dfml ccwj"

with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.ehlo()
    server.starttls()
    server.login(smtp_user, smtp_pass)
    server.send_message(msg)

print("Email sent successfully!")


Email sent successfully!
